# Hnefatafl - AlphaZero
## CISC-856 W21 Project
## Written by Tindur Sigurdason and Calen Irwin

##Purpose: 

Combine all project components to demonstrate our AlphaZero-based algorithm's ability to learn Hnefatafl through self play

##Instructions:
- Upload this notebook to a cloud environment like Google Colab 
- Click 'Runtime > Run all' and wait for runtime to restart
- Click the first cell under Load Libraries and then click 'Runtime > Run after' in order to execute all of the remaining cells in the notebook

In [11]:
!rm -rf 'gym-hnef'

In [12]:
!git clone https://github.com/calenirwin/gym-hnef
!pip install -e gym-hnef

Cloning into 'gym-hnef'...
remote: Enumerating objects: 1122, done.
remote: Counting objects: 100% (529/529), done.
remote: Compressing objects: 100% (358/358), done.
remote: Total 1122 (delta 346), reused 348 (delta 171), pack-reused 593
Receiving objects: 100% (1122/1122), 5.63 MiB | 36.73 MiB/s, done.
Resolving deltas: 100% (719/719), done.
Obtaining file:///content/gym-hnef
  Found existing installation: gym-hnef 0.0.1
    Can't uninstall 'gym-hnef'. No files were found to uninstall.
  Running setup.py develop for gym-hnef


In [ ]:
import os

# restarts the runtime after download github repo and install the custom gym env
def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

# Load Libraries

In [4]:
# load necessary libraries
# highlight this cell and click 'Runtime > Run after', once you have install the 
# gym env and restarted the runtime

# external libraries
import gym
import numpy as np
from matplotlib import pyplot as plt

# local libraries
from gym_hnef import hnef_game, hnef_vars
from gym_hnef.envs import hnef_env
import agent, config, funcs, mcts, memory, model

In [5]:
#action_size = 6561 # historical action space => 9^4
action_size = 625   # mini action space => 5^4

# intialize game state
#init_state = hnef_game.init_state('historical')
init_state = hnef_game.init_state('mini')

In [6]:
# instantiate the neural networks
current_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)
best_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)

In [7]:
# instantiate the game agents
current_player = agent.Agent('current_player', current_NN, init_state.shape, action_size)
best_player = agent.Agent('best_player', best_NN, init_state.shape, action_size)

In [8]:
iteration = 1
max_iter = 20
mem = memory.Memory(config.MEMORY_SIZE)
version = 0

while iteration < max_iter:
    print('Iteration number:', iteration)

    # self play
    scores, mem = funcs.play_matches(best_player, best_player, mem=mem, rule_set='mini')
    print(scores)

    mem.clear_stmemory()

    # if the memory has exceeded its capacity, then initiate a tournament to decide the best player
    if len(mem.ltmemory) >= config.MEMORY_SIZE:

        current_player.replay(mem.ltmemory)

        # tournament between best and current agent
        scores, _ = funcs.play_matches(best_player, current_player, rule_set='mini')
        print(scores)

        # if the current player wins more than the given threshold, reassign the best player to the current player
        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            version += 1
            print(version)
            # changes weights to the new best player's weights
            best_NN.model.set_weights(current_NN.model.get_weights())
    else:
        print('Memory size', str(len(mem.ltmemory)))
    iteration += 1

Iteration number: 1
Starting Episode  1 ...
Episode Completed. Final Game State:
     0   1   2   3   4
  ---------------------
0 |   | A |   |   | A |
  ---------------------
1 | A |   | A |   | A |
  ---------------------
2 | K |   |   |   |   |
  ---------------------
3 | D |   |   | D |   |
  ---------------------
4 | A |   |   | A | A |
  ---------------------
	Turn: Attacker
Defender win
Starting Episode  2 ...
Episode Completed. Final Game State:
     0   1   2   3   4
  ---------------------
0 | A |   |   |   |   |
  ---------------------
1 | D | A |   | A |   |
  ---------------------
2 |   |   |   |   | A |
  ---------------------
3 |   | D |   | A |   |
  ---------------------
4 | A | A |   | A |   |
  ---------------------
	Turn: Defender
Attacker win
Starting Episode  3 ...
Episode Completed. Final Game State:
     0   1   2   3   4
  ---------------------
0 | A | A |   | A |   |
  ---------------------
1 |   |   |   | D | A |
  ---------------------
2 |   |   |   |   | K 

In [ ]:
random_player = agent.RandomAgent('random_player', init_state.shape, action_size)
scores, all_end_states = funcs.evaluate_agents(best_player, random_player, num_games=500, rule_set='mini', switch_sides=True)

Streaming output truncated to the last 5000 lines.
  ---------------------
4 | A |   |   | A |   |
  ---------------------
	Turn: Attacker
Defender win
Game 2 Final State:
    0   1   2   3   4
  ---------------------
0 |   |   |   |   |   |
  ---------------------
1 |   |   | A | A | A |
  ---------------------
2 | A |   |   |   | K |
  ---------------------
3 |   | D | D | D | A |
  ---------------------
4 | A | A |   | A |   |
  ---------------------
	Turn: Attacker
Defender win
Game 3 Final State:
    0   1   2   3   4
  ---------------------
0 |   | A |   | A |   |
  ---------------------
1 |   | D |   |   | A |
  ---------------------
2 | A | D |   |   | D |
  ---------------------
3 |   | A |   | A | A |
  ---------------------
4 |   | A | K |   |   |
  ---------------------
	Turn: Attacker
Defender win
Game 4 Final State:
    0   1   2   3   4
  ---------------------
0 |   |   |   |   |   |
  ---------------------
1 | A | A |   |   | A |
  ---------------------
2 | A |   |   | 